In [16]:
pip install geopy folium pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [25]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
import webbrowser  # Add this import

# Load the data from Excel
df = pd.read_excel('Biggest football stadiums in wales.xlsx')


In [26]:
df.head()

,capacity,stadium,postcode,team(s),latitude and longitude
0,10500,cwmbran stadium,NP44 3YS,Cwmbran Town,"51.6427° N, 3.0214° W"
1,6000,Jenner Park stadium,CF62 9BG,"Barry Town United, Barry Town United Women","51.4114° N, 3.2680° W"
2,6000,The Remax Stadium,LL11 2AH,"Port Talbot Town, Port Talbot Town Ladies","53.0516° N, 3.0043° W"
3,5078,Newport Stadium,NP19 0UU,Newport City,"51.5896° N, 2.9913° W"
4,5000,Latham Park,CW15 1AE,\tNewtown,"53.155° N, 2.458° W"


In [27]:
# Clean the data - remove empty rows
df = df.dropna(how='all')

# Function to extract latitude and longitude from the string
def extract_lat_lon(coord_str):
    try:
        if pd.isna(coord_str):
            return None, None
        parts = coord_str.split(',')
        lat_part = parts[0].strip()
        lon_part = parts[1].strip()
        
        lat = float(lat_part.split('°')[0])
        if 'S' in lat_part:
            lat = -lat
            
        lon = float(lon_part.split('°')[0])
        if 'W' in lon_part:
            lon = -lon
        elif 'E' in lon_part:
            pass  # lon is already positive
        else:
            # Handle cases where direction isn't specified but value is negative
            if float(lon_part.split('°')[0]) < 0:
                lon = float(lon_part.split('°')[0])
            
        return lat, lon
    except:
        return None, None

In [28]:
# Apply the function to create latitude and longitude columns
df['latitude'], df['longitude'] = zip(*df['latitude and longitude'].apply(extract_lat_lon))

# Drop rows where we couldn't get coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# Create a map centered on Wales
wales_map = folium.Map(location=[52.1307, -3.7837], zoom_start=8)

In [29]:
# Add markers for each stadium
for idx, row in df.iterrows():
    popup_text = f"""
    <b>Stadium:</b> {row['stadium']}<br>
    <b>Capacity:</b> {row['capacity']}<br>
    <b>Team(s):</b> {row['team(s)']}<br>
    <b>Postcode:</b> {row['postcode']}
    """
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['stadium']
    ).add_to(wales_map)

In [30]:
# Save the map to an HTML file
wales_map.save('welsh_football_stadiums.html')

print("Map created successfully! Open 'welsh_football_stadiums.html' in your browser.")

Map created successfully! Open 'welsh_football_stadiums.html' in your browser.


In [ ]:
# Save the map
map_file = 'welsh_football_stadiums.html'
wales_map.save(map_file)

# Open in default browser
webbrowser.open(map_file)

True